# Download script

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse


base_url = 'https://www.digitalrocksportal.org/projects/372'
download_directory = 'downloads'

os.makedirs(download_directory, exist_ok=True)

def download_file(url, directory, filename):
    if not filename:
        parsed_url = urlparse(url)
        filename = os.path.basename(parsed_url.path)

    if not filename:
        filename = 'downloaded_file'

    local_filename = os.path.join(directory, filename)


    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(local_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    return local_filename

def fetch_links(url, keyword):
    """Функция для извлечения ссылок по ключевому слову"""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [
        urljoin(url, a['href']) for a in soup.find_all('a', href=True)
        if keyword in a['href']
    ]
    return links


def process_file_links(file_links_soup, directory, name):
    for div in file_links_soup.find_all('div', class_='dataset-file'):
        h4 = div.find('h4')
        if h4 and  h4['title'][0].isdigit():
            title = h4['title']
        else:
            title = name+"_"+h4['title']
        print(h4['title'])

        download_link = [a['href'] for a in div.find_all('a', href=True) if "download" in a['href'] and "downloadMeta" not in a['href']]
        print(download_link[0])
        if download_link[0]:
            file_url = urljoin(base_url, download_link[0])
            print(f"Скачиваю файл: {file_url} как {title or 'default'}")
            download_file(file_url, directory, title)


def process_analysis_data(analysis_data_url):
    print(f"Обрабатываю origin_data: {analysis_data_url}")


    response = requests.get(analysis_data_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    names = [[j for j in i.get_text(strip=False).split() if j != "\n" and j[0].isdigit()][0] for i in soup.find_all('h2')]
    print(names[0])

    process_file_links(soup, download_directory, names[0])



def process_origin_data(origin_data_url):
    print(f"Обрабатываю origin_data: {origin_data_url}")


    response = requests.get(origin_data_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    names = [[j for j in i.get_text(strip=False).split() if j != "\n" and j[0].isdigit()][0] for i in soup.find_all('h2')]
    print(names[0])

    process_file_links(soup, download_directory, names[0])

    analysis_data_links = fetch_links(origin_data_url, 'analysis_data')

    for analysis_data_url in analysis_data_links:
        process_analysis_data(analysis_data_url)

def process_sample(sample_url):
    print(f"Обрабатываю сэмпл: {sample_url}")


    origin_data_links = fetch_links(sample_url, 'origin_data')

    for origin_data_url in origin_data_links:
        process_origin_data(origin_data_url)

def main():

    sample_links = fetch_links(base_url, 'sample')


    for sample_url in sample_links:
        process_sample(sample_url)

if __name__ == '__main__':
    main()

In [ ]:
import sys

import random
import struct
import time
from array import array
from datetime import timedelta
from os.path import join

import cv2
import gudhi as gd
import matplotlib.pyplot as plt
import numpy as np

# from src.persistence import sublevel_persistence
# from src.models import Transformer, ConvolutionalPersistenceHomologyTransformer, DirectionalPersistenceHomologyTransformer
import pandas as pd
import porespy as ps
import scipy as sp
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms.v2 import Compose, Lambda
from tqdm import tqdm
import math
import pickle
from typing import Any, Dict

import numpy as np
import quantimpy
import torch
import torch.nn as nn
from scipy.ndimage import distance_transform_edt, grey_dilation
from torchvision.transforms.v2 import Transform
import os
import sys
from urllib.request import urlretrieve
import glob
import pandas as pd
from collections import defaultdict
from hdf5storage import loadmat

import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import tensorflow as tf
import tensorflow.keras.backend as kb
from scipy.ndimage import distance_transform_edt as distance
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D
from tensorflow.keras.models import Model

np.set_printoptions(precision=3, linewidth=120, edgeitems=45, threshold=100)
torch.set_printoptions(precision=3, linewidth=120, edgeitems=45, threshold=100)

In [ ]:
def getName(filename):
    dl = 10
    while filename[dl:dl +3] != "256" and filename[dl:dl +3] != "480":
        dl += 1
    return filename[:dl +3]

In [ ]:
def calculate_diagramm(image):
    distances = distance_transform_edt(image == 0) - distance_transform_edt(image == 1)
    comp = gd.CubicalComplex(top_dimensional_cells = distances)
    return comp.persistence(homology_coeff_field=19, min_persistence=0)

In [ ]:
y = []
file_names = []
distances = []
mat_datas = []
fils = glob.glob("downloads/*")
fils.sort()
i = 0

while i < len(fils):
    has_y = False
    has_data = False
    has_pkl = False
    name0 = getName(fils[i])
    while i < len(fils) and getName(fils[i]) == name0:
        if fils[i].endswith("_P_1_MPa.csv"):
            df = pd.read_csv(fils[i], sep=",", header=None)
            has_y = True
        if fils[i].endswith(".pkl"):
            has_pkl = True
            fname = fils[i]
            
        if getName(fils[i]) + ".mat" == fils[i] and getName(fils[i])[-3:] == '256':
            print(getName(fils[i]) + ".mat")
            try:
                mat_data = loadmat(fils[i])["bin"]
                dist = -distance_transform_edt(mat_data==1) + distance_transform_edt(mat_data==0)
                has_data = True
            except KeyError:
                has_data = False


        i += 1
    if (has_pkl and has_y and has_data): #and has_data
        y.append(df.iloc[4].values[-1])
        file_names.append(fname)
        distances.append(dist)
        mat_datas.append(mat_data)

In [ ]:
pickle.dump(mat_datas, open("./natures_images", "wb"))
pickle.dump(y, open("./y_natutes", "wb"))

# Calculate curves

In [ ]:
np_images_dist = np.array(distances)
thresholds = np.linspace(np_images_dist.min(), np_images_dist.max(), num=50)

In [ ]:
import pickle

def calculate_betti(filename, thresholds):
      with open(filename, "rb") as f:
        persistence = pickle.load(f)

        all_events = []

        dots = [[], [], []]

        for item in persistence:
            all_events += [(item[1][0], +1, item[0])]
            all_events += [(item[1][1], -1, item[0])]
            dots[item[0]].append(item[1])

        all_events.sort()

        betti = []
        betti_current = [0, 0, 0]

        threshold_index = 0

        for event in all_events:
            t, delta, dim = event
            while threshold_index < len(thresholds) and t >= thresholds[threshold_index]:
                betti.append(betti_current[:])
                threshold_index += 1
            betti_current[int(dim)] += delta
        return (betti, dots)

In [ ]:
betti_curves = []
for file in file_names:
    betti_curves.append(calculate_betti(file, thresholds)[0])
    dots = calculate_betti(file, thresholds)[1]

In [ ]:
minkowski_curves = []
from quantimpy import minkowski

for dis in distances:
    curve = []
    for thr in thresholds:
        curve.append(minkowski.functionals(np.ascontiguousarray(dis < thr)))
    minkowski_curves.append(curve)

In [ ]:
features = np.dstack([np.array(minkowski_curves), np.array(betti_curves)])
target = np.array(y)

In [ ]:
pickle.dump(features, open("./natures_minkovsky+betti", "wb"))

# calculating Diagrams 

In [ ]:
def direction_filter3d(img, alpha, beta):
    width, height, depth = img.shape
    
    # Центрирование по каждой оси (для координат относительно центра)
    X = (np.cos(alpha)*np.sin(beta) - (np.arange(width)  - (width/2  - 0.5)) / (width  * np.sqrt(3))) * np.cos(alpha)*np.sin(beta) / 2
    Y = (np.sin(alpha)*np.sin(beta) - (np.arange(height) - (height/2 - 0.5)) / (height * np.sqrt(3))) * np.sin(alpha)*np.sin(beta) / 2
    Z = (np.cos(beta)              - (np.arange(depth)  - (depth/2  - 0.5)) / (depth  * np.sqrt(3))) * np.cos(beta)              / 2
    
    # Складываем, как в 2D варианте!
    direction_filter = (
        X.reshape(-1, 1, 1) +  # вес по оси X повторяется для всех Y,Z
        Y.reshape(1, -1, 1) +  # вес по оси Y повторяется для всех X,Z
        Z.reshape(1, 1, -1)    # вес по оси Z повторяется для всех X,Y
    )
    return torch.Tensor(np.multiply(direction_filter, img))



class Direction(Transform):
    """Transform an image with the direction transform."""
    def __init__(self, alphas):
        super().__init__()
        self.alphas = alphas
        self.weight = alphas

    def transform3d(self, inpt: Any, params: Dict[str, Any]) -> Any:
        output = torch.zeros(len(self.alphas), inpt.shape[-3], inpt.shape[-2], inpt.shape[-1])
        for i, alpha in enumerate(self.alphas):
            output[i] = direction_filter3d(inpt, alpha[0], alpha[1])
        return output


In [ ]:
def pht(image, pos=None, eps=None):

    if len(image.shape) != 4:
        raise ValueError("Image tensor dimensions should be (channel, width, height).")

    dim = 6 if pos is not None else 3
    dgm_pht = torch.zeros((0, dim))
    dgms = sublevel_persistence(image, eps=eps, pos=pos, sort="persistence")

    for i, dgm in enumerate(dgms):
        dgm_pht = torch.cat([dgm_pht, dgm])

    return dgm_pht


def sublevel_persistence(image, eps=None, pos=None, inf="max", sort="birth"):

    if len(image.shape) != 4:
        raise ValueError("Image tensor dimensions should be (channel, width, height).")

    diagrams = []
    persistence = lambda x: x[:, 1] - x[:, 0]

    # for each channel
    for k, channel in enumerate(image):
        diagram_channel_gudhi = gd.CubicalComplex(
            top_dimensional_cells=channel
        ).persistence()

        # convert a diagram from GUDHI format to n x 3 ndarray
        diagram_channel = np.zeros((len(diagram_channel_gudhi), 3))
        for i, (dim, (birth, death)) in enumerate(diagram_channel_gudhi):
            diagram_channel[i] = (birth, death, dim)

        # work with infs
        if inf == "max":
            diagram_channel = np.nan_to_num(diagram_channel, posinf=torch.max(channel))
        elif inf == "remove":
            diagram_channel = diagram_channel[~np.isinf(diagram_channel).any(axis=1)]
        else:
            raise ValueError("Inf should be 'max' or 'remove'.")

        # remove points w/ persistence less \eps
        if eps is not None:
            diagram_channel = diagram_channel[persistence(diagram_channel) > eps]

        # add positional encoding
        if pos is not None:
            pos_elements1 = np.repeat(pos[k][0], len(diagram_channel))[..., np.newaxis]
            pos_elements2 = np.repeat(pos[k][1], len(diagram_channel))[..., np.newaxis]
            pos_idx = np.repeat(k, len(diagram_channel))[..., np.newaxis]
            diagram_channel = np.concatenate(
                [diagram_channel, pos_elements1, pos_elements2, pos_idx], axis=1
            )

        # sort by dim, then birth or persistence
        if sort == "birth":
            sort_idx = np.lexsort([diagram_channel[:, 0], diagram_channel[:, 2]])
        elif sort == "persistence":
            sort_idx = np.lexsort([persistence(diagram_channel), diagram_channel[:, 2]])
        else:
            raise ValueError("Sort should be 'birth' or 'persistence'.")
        diagram_channel = diagram_channel[sort_idx]

        diagrams.append(torch.tensor(diagram_channel))

    return diagrams

In [ ]:
import torch
from scipy.ndimage import distance_transform_edt
from torchvision.transforms.v2 import Compose, ToDtype

transform_rock_image = Compose(
    [
        ToDtype(torch.float32),
    ]
)


alphas = np.linspace(0, 360, 6, endpoint=True)
betas = np.linspace(0, 180, 3, endpoint=True)
pairs_deg = np.stack([np.repeat(alphas, 3), np.tile(betas, 6)], axis=1)
pairs_deg = pairs_deg / 180.0 * np.pi

In [ ]:
func = Direction(pairs_deg)


def get_diagram_filtered(image):
    image_sedt = -distance_transform_edt(image == 0) + distance_transform_edt(image == 1)
    ingt = torch.from_numpy(image_sedt)
    imgt = transform_rock_image(ingt)
    image_filtered = func.transform3d(imgt, [])
    imgt = imgt[None]
    diagram = pht(image_filtered, pos=pairs_deg, eps=0.05)
    diagram_filterd = pht(imgt, eps=0.05)
    return diagram, diagram_filterd

In [ ]:
import pickle

images = pickle.load(open("/home/jupyter/datasphere/project/natures_images", "rb"))

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm
import pickle

def process_image(im):
    diagram, diagram_filterd = get_diagram_filtered(im)
    return diagram, diagram_filterd


diagrams_filtered = []
diagrams = []

with Pool(processes=16) as pool:   
    results = list(tqdm(
        pool.imap(process_image, images),
        total=len(images)
    ))

diagrams_filtered, diagrams = zip(*results)
diagrams_filtered = list(diagrams_filtered)
diagrams = list(diagrams)


In [ ]:
filename = f'./natures_diagrams_filtered.pkl'
pickle.dump(diagrams_filtered, open(filename, "wb"))
filename = f'./diagram_natures.pkl'
pickle.dump(diagrams, open(filename, "wb"))